In [1]:
import pandas as pd

df = pd.read_csv("BarcelonaAbnb_limpio.csv")
df
import numpy as np

df["log_price"] = np.log1p(df["price"])


In [2]:
y_reg = df["price"].values
y_reg


array([210., 285., 170., ...,  90.,  90., 120.])

In [3]:
cols_excluir_reg = [
    "price",
    "estimated_revenue_l365d"
    ,  "log_price"
]

X_reg = df.drop(columns=cols_excluir_reg)
X_reg = X_reg.select_dtypes(include=["number"])

X_reg 


,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,...,room_type_Shared room,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi
0,0.0,96.0,91.0,0.0,1.0,1.0,41.405560,2.172620,8.0,2.0,...,0,1,0,0,0,0,0,0,0,0
1,0.0,100.0,96.0,1.0,1.0,1.0,41.412432,2.219750,5.0,2.0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,100.0,100.0,0.0,1.0,1.0,41.405660,2.170150,6.0,2.0,...,0,0,1,0,0,0,0,0,0,0
3,0.0,100.0,92.0,0.0,1.0,1.0,41.380620,2.175170,2.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,100.0,92.0,0.0,1.0,1.0,41.379780,2.176230,9.0,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13410,1.0,100.0,98.0,0.0,1.0,1.0,41.382010,2.177120,5.0,1.0,...,0,0,0,0,0,0,0,0,0,0
13411,1.0,87.0,6.0,0.0,1.0,1.0,41.410689,2.170413,3.0,1.0,...,0,0,0,1,0,0,0,0,0,0
13412,1.0,87.0,6.0,0.0,1.0,1.0,41.390695,2.164782,4.0,1.0,...,0,1,0,0,0,0,0,0,0,0
13413,1.0,87.0,6.0,0.0,1.0,1.0,41.396987,2.166294,4.0,1.0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=42
)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)


In [6]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear") 
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [7]:
n_features_reg = X_train_reg_scaled.shape[1]

model_reg = build_regression_model(n_features_reg)

history_reg = model_reg.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 8305.8389 - mae: 62.9349 - val_loss: 3275.1367 - val_mae: 41.4160
Epoch 2/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3336.2422 - mae: 41.4538 - val_loss: 3038.8176 - val_mae: 40.1804
Epoch 3/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3135.7705 - mae: 39.9749 - val_loss: 2925.2815 - val_mae: 38.6039
Epoch 4/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3100.3394 - mae: 39.5384 - val_loss: 2856.0815 - val_mae: 38.5295
Epoch 5/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3024.9380 - mae: 39.1410 - val_loss: 2820.3457 - val_mae: 38.5723
Epoch 6/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2942.8643 - mae: 38.7092 - val_loss: 2773.7166 - val_mae: 37.9574
Epoch 7/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2899.2544 - mae: 38.3581 - val_loss: 2784.0046 - val_mae: 38.0939
Epoch 8/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2896.7419 - mae: 38.2727 - val_loss: 2792.4207 - v

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred_reg = model_reg.predict(X_test_reg_scaled).flatten()

mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_reg, y_pred_reg)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)



84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step
MSE: 2780.9194715110425
RMSE: 52.73442397060048
MAE: 36.200376147349466


In [9]:
df["price"].describe()

eval = rmse/df["price"].mean()
print("Evaluation (RMSE / Mean Price):", eval)


Evaluation (RMSE / Mean Price): 0.3659523133066816


Hicimos research y vimos que transformar logarítmicamente el precio estabiliza la varianza y reduce la influencia de outliers, lo que nos puede servir para predecir de mejor forma. Ahora predeciremos con base en esto.

In [10]:
y_reg = df["log_price"].values


In [11]:
cols_excluir_reg = [
    "price",
    "log_price",
    "estimated_revenue_l365d"  
]

X_reg = df.drop(columns=cols_excluir_reg)
X_reg = X_reg.select_dtypes(include=["number"])


In [12]:
from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=42
)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)


In [14]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [15]:
n_features_reg = X_train_reg_scaled.shape[1]

model_reg_log = build_regression_model(n_features_reg)

history_reg_log = model_reg_log.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.1349 - mae: 1.0096 - val_loss: 0.3382 - val_mae: 0.4499
Epoch 2/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5530 - mae: 0.5870 - val_loss: 0.3388 - val_mae: 0.4642
Epoch 3/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4175 - mae: 0.5107 - val_loss: 0.2212 - val_mae: 0.3628
Epoch 4/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3389 - mae: 0.4575 - val_loss: 0.2545 - val_mae: 0.4086
Epoch 5/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2856 - mae: 0.4203 - val_loss: 0.2428 - val_mae: 0.3990
Epoch 6/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2546 - mae: 0.3996 - val_loss: 0.2296 - val_mae: 0.3874
Epoch 7/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2167 - mae: 0.3646 - val_loss: 0.1892 - val_mae: 0.3467
Epoch 8/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2009 - mae: 0.3522 - val_loss: 0.3523 - val_mae: 0.5013
Epoch 9/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

In [16]:
y_pred_log = model_reg_log.predict(X_test_reg_scaled).flatten()

# Convertir log_price → price
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test_reg)


84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

MSE: 2323.4104729739647
RMSE: 48.20176835940736
MAE: 31.88671233837968


In [24]:
import mlflow
import mlflow.keras

mlflow.set_tracking_uri("http://localhost:5000") 
experiment_reg = mlflow.set_experiment("airbnb-barcelona-reg-nn")
print("Experiment ID (reg):", experiment_reg.experiment_id)



2025/11/30 22:08:17 INFO mlflow.tracking.fluent: Experiment with name 'airbnb-barcelona-reg-nn' does not exist. Creating a new experiment.


Experiment ID (reg): 614581612292287758


In [25]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear")  # regresión
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [26]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_mlflow_reg(
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3,
    epochs=80,
    batch_size=32,
    run_name="reg_base",
    experiment_id=None
):
    n_features_reg = X_train_reg_scaled.shape[1]

    with mlflow.start_run(run_name=run_name, experiment_id=experiment_id):

        # Log hiperparámetros
        mlflow.log_param("n_hidden1", n_hidden1)
        mlflow.log_param("n_hidden2", n_hidden2)
        mlflow.log_param("n_hidden3", n_hidden3)
        mlflow.log_param("dropout_rate", dropout_rate)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

        # Construir modelo
        model = build_regression_model(
            n_features=n_features_reg,
            n_hidden1=n_hidden1,
            n_hidden2=n_hidden2,
            n_hidden3=n_hidden3,
            dropout_rate=dropout_rate,
            learning_rate=learning_rate
        )

        # Entrenar
        history = model.fit(
            X_train_reg_scaled,
            y_train_reg,
            validation_data=(X_test_reg_scaled, y_test_reg),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0
        )

        # Predicción en escala log
        y_pred_log = model.predict(X_test_reg_scaled).flatten()

        # Volver a precio real (euros)
        y_true = np.expm1(y_test_reg)
        y_pred = np.expm1(y_pred_log)

        # Métricas reales
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        mean_price = y_true.mean()
        rel_rmse = rmse / mean_price

        # Log métricas
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("rmse_over_mean_price", rel_rmse)

        # Guardar modelo
        mlflow.keras.log_model(model, "model")

        print(f"Run '{run_name}' -> RMSE={rmse:.2f}, MAE={mae:.2f}, RMSE/Mean={rel_rmse:.3f}")


In [27]:
configs_reg = [
    # ---- MODELOS PEQUEÑOS ----
    {"run_name": "reg_small_1", "n_hidden1": 32,  "n_hidden2": 16,  "n_hidden3": 8,   "learning_rate": 1e-3, "dropout_rate": 0.1, "epochs": 60,  "batch_size": 32},
    {"run_name": "reg_small_2", "n_hidden1": 32,  "n_hidden2": 16,  "n_hidden3": 8,   "learning_rate": 5e-4, "dropout_rate": 0.1, "epochs": 80,  "batch_size": 32},
    {"run_name": "reg_small_3", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16,  "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 50,  "batch_size": 64},
    {"run_name": "reg_small_4", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 1e-3, "dropout_rate": 0.0, "epochs": 50,  "batch_size": 32},
    {"run_name": "reg_small_5", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 5e-4, "dropout_rate": 0.1, "epochs": 70,  "batch_size": 32},

    # ---- MODELOS MEDIOS ----
    {"run_name": "reg_medium_1", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 80,  "batch_size": 32},
    {"run_name": "reg_medium_2", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 5e-4, "dropout_rate": 0.3, "epochs": 90,  "batch_size": 32},
    {"run_name": "reg_medium_3", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64, "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 70,  "batch_size": 64},
    {"run_name": "reg_medium_4", "n_hidden1": 64,  "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 3e-4, "dropout_rate": 0.2, "epochs": 90,  "batch_size": 32},
    {"run_name": "reg_medium_5", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-4, "dropout_rate": 0.2, "epochs": 100, "batch_size": 32},

    # ---- MODELOS GRANDES ----
    {"run_name": "reg_large_1", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 1e-3, "dropout_rate": 0.3, "epochs": 100, "batch_size": 32},
    {"run_name": "reg_large_2", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 5e-4, "dropout_rate": 0.3, "epochs": 120, "batch_size": 32},
    {"run_name": "reg_large_3", "n_hidden1": 512, "n_hidden2": 256, "n_hidden3": 128, "learning_rate": 3e-4, "dropout_rate": 0.3, "epochs": 120, "batch_size": 64},
    {"run_name": "reg_large_4", "n_hidden1": 512, "n_hidden2": 256, "n_hidden3": 128, "learning_rate": 1e-4, "dropout_rate": 0.4, "epochs": 140, "batch_size": 32},
    {"run_name": "reg_large_5", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 2e-4, "dropout_rate": 0.3, "epochs": 140, "batch_size": 32},

    # ---- VARIANDO LEARNING RATE ----
    {"run_name": "reg_lr_1e_minus2", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16, "learning_rate": 1e-2, "dropout_rate": 0.2, "epochs": 40,  "batch_size": 32},
    {"run_name": "reg_lr_5e_minus3", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16, "learning_rate": 5e-3, "dropout_rate": 0.2, "epochs": 50,  "batch_size": 32},
    {"run_name": "reg_lr_3e_minus4", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 3e-4, "dropout_rate": 0.2, "epochs": 100, "batch_size": 32},
    {"run_name": "reg_lr_1e_minus4", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-4, "dropout_rate": 0.3, "epochs": 120, "batch_size": 32},
    {"run_name": "reg_lr_5e_minus5", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64, "learning_rate": 5e-5, "dropout_rate": 0.2, "epochs": 150, "batch_size": 32},
]


In [28]:
for cfg in configs_reg:
    print(f"=== Corriendo {cfg['run_name']} ===")
    run_mlflow_reg(**cfg, experiment_id=experiment_reg.experiment_id)
    print()


=== Corriendo reg_small_1 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step


2025/11/30 22:09:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 22:09:14 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/30 22:09:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 'reg_small_1' -> RMSE=54.35, MAE=36.72, RMSE/Mean=0.378
🏃 View run reg_small_1 at: http://localhost:5000/#/experiments/614581612292287758/runs/bf7d5f3da9ef49058ff116a4da179496
🧪 View experiment at: http://localhost:5000/#/experiments/614581612292287758

=== Corriendo reg_small_2 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step


2025/11/30 22:10:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 22:10:01 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/30 22:10:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 'reg_small_2' -> RMSE=58.34, MAE=39.28, RMSE/Mean=0.406
🏃 View run reg_small_2 at: http://localhost:5000/#/experiments/614581612292287758/runs/64a3b47880ea471689e10608028d8463
🧪 View experiment at: http://localhost:5000/#/experiments/614581612292287758

=== Corriendo reg_small_3 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


...

Run 'reg_lr_3e_minus4' -> RMSE=53.12, MAE=35.13, RMSE/Mean=0.369
🏃 View run reg_lr_3e_minus4 at: http://localhost:5000/#/experiments/614581612292287758/runs/7a4f4aa92bdd43b0a7e73d6bbd34dfdf
🧪 View experiment at: http://localhost:5000/#/experiments/614581612292287758

=== Corriendo reg_lr_1e_minus4 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step


2025/11/30 22:28:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 22:28:13 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/30 22:28:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 'reg_lr_1e_minus4' -> RMSE=59.55, MAE=39.95, RMSE/Mean=0.414
🏃 View run reg_lr_1e_minus4 at: http://localhost:5000/#/experiments/614581612292287758/runs/2bbd7a21825a4cf3a05f0436dcdb1935
🧪 View experiment at: http://localhost:5000/#/experiments/614581612292287758

=== Corriendo reg_lr_5e_minus5 ===
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step


2025/11/30 22:30:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 22:30:20 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/30 22:30:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 'reg_lr_5e_minus5' -> RMSE=54.69, MAE=36.40, RMSE/Mean=0.380
🏃 View run reg_lr_5e_minus5 at: http://localhost:5000/#/experiments/614581612292287758/runs/fd944968e3ca410a900df6d20d4cbbb5
🧪 View experiment at: http://localhost:5000/#/experiments/614581612292287758

